# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
#from datasets import load_dataset
from datasets import load_from_disk

#dataset = load_dataset("yelp_review_full")
dataset = load_from_disk('/root/autodl-tmp/yelp_review_full')

/root/miniconda3/envs/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,2 star,"A bunch of us thought we'd try somewhere new for lunch and walked away unimpressed.\n\nThe first oddity was walking through the front door that was propped open...in the middle of the day in Arizona...in July. Then we got to listen to the bartender discuss how to smoke marijuana with someone at the bar for a few minutes before tending to our table. All of the food was just off. The Penang Curry tasted like it was mixed with a gallon of coconut milk, the Pad Thai tasted like orange chicken, my Pad Woon-Sen had 3 or 4 small pieces of chicken and a mash up of under-cooked veggies. The guy that ordered a coke had his drink refilled from a pitcher that had been sitting on the bar long before we arrived. When asked about getting flat, warm soda we were told that \""its all about the efficiency\"". My experience was capped by not getting any change from the cash I gave to pay the bill. \n\nUnderwhelmed all around."
1,5 stars,"For a designer addict like me, the Forum Shops are a dream come true. While we have a lot of the same stores here in Scottsdale, there's something so posh about walking through the forum shops, popping into Louis and Burberry and Marc Jacobs, filling your arms with bag after bag, and making your man hang on to some because you couldn't possibly carry them off. By far and away, my favorite place to go broke.\n\nStop by: Spago, the Gelato place, Marc Jacobs & the MAC store (that carries the regular and PRO line!)"
2,1 star,Wish I had known what a douche pit this place would be before we booked.
3,3 stars,Can never go wrong. Parking is kinda hard sometimes. No big issues. It's close so we stop by often.
4,4 stars,"Definitely try to get a seat by the fountain out on the patio. Let's put it this way, they have weird, funny, somewhat entertaining things going on while you eat. Call days in advance for a reservation!!\n\nI LOVE their steak!!! So why the 4 stars? First, the service was not stellar. I think we just got a bad waiter during our last visit (the first time we went a year ago, we had amazing service). He didn't care to give us drink menus, and when we asked we got only one drink menu and not the wine list. He didn't care to give us bread until we asked. He rarely came by at all. We had other people come by pouring us water and things but honestly, I think we only saw him again when it came time to pay the bill. I totally expect more from an expensive steakhouse like this one.\n\nSecond, their side dishes are amazing....but their mac & cheese sucked. I can't believe I'm saying this, but it could be the worst mac & cheese I have ever eaten at a high end restaurant like this. Boo. And I love mac & cheese.\n\nWith that said, like I said, I LOVE their steak. I like it better than The Prime, Sinatra, STK and maybe even Craft (but I really did love Craft too).\n\nDespite their mediocre service, which I think was a fluke, and their not-so-hot mac & cheese, I will DEFINITELY be back. Their steaks are that good."
5,1 star,"Some of the most delectable sweet treats I have had in Las Vegas have come from Pinkbox. BUT the experience I had there recently pretty much negates all of that. I ordered a dozen, and got a mix of classic and fancy schmancy gourmet. To be specific:\n3 glazed\n3 croissant style w/ cinnamon and sugar\n1 smores\n2 Maple Bacon\n2Fat Elvis\n1 Apple pie\nThey were SWAMPED! I got my box of goodies, and left. Now when I got them the glazed were still warm, and I THOUGHT that was a good thing. I was so wrong. When I got to the office and everyone dug in we realized the glazed AND the cronuts were RAW. Not kinda doughy- straight up not cooked. I was surprised, and kinda grossed out, so I called. The person I spoke to did not give there name, but identified themselves as the manager. He told me that I was wrong, was rude, dismissive, and with a rude little quip about \""not having time for this\""-hung up. I followed up with an email, and never heard back. It left a really bad taste in my

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/bert-base-cased-model", local_files_only=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=512, padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:12<00:00, 3982.51 examples/s]


In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"I don't know what I was thinking when I decided to eat here. I should have asked myself this question before entering how could a candy store possibly be skilled at making food? They even failed at making good dessert, which is surprising since they are called the Sugar Factory American Brasserie. SUGAR is in the name of the restaurant how could they mess up on that?\n\nI got the frozen hot chocolate. Powdery and not exciting at all this drink is not worth ordering. The appetizers my table ordered were just \""ehh\"". The spinach and artichoke dip was served with pita chips. The dip itself was sour and we only finished it because we were starving.\n\nService was whatever. Our waitress lagged in giving us our dessert. She came over to our table and when we asked about our dessert she had a surprised look on her face and admitted to forgetting on putting in the order. Dessert was the most disappointing part. I don't know if it's because we had to wait a long time for it but the chocolate fondue? It's just a tiny pot of melted chocolate kept warm by a little tea candle (the candle surely wasn't hot enough to keep the chocolate in a liquid state). The toppings that accompanied it were interesting... gummy bears? In the future I think i'll go to Melting Pot for my chocolate fondue.\n\nThis place oddly reminds me of Cheesecake Factory but a little pricier and with less appetizing food.","[101, 146, 1274, 112, 189, 1221, 1184, 146, 1108, 2422, 1165, 146, 1879, 1106, 3940, 1303, 119, 146, 1431, 1138, 1455, 1991, 1142, 2304, 1196, 5273, 1293, 1180, 170, 13962, 2984, 3566, 1129, 10715, 1120, 1543, 2094, 136, 1220, 1256, 2604, 1120, 1543, 1363, 20392, 117, 1134, 1110, 11567, 1290, 1152, 1132, 1270, 1103, 10619, 9977, 1237, 20982, 15148, 119, 156, 2591, 10583, 2069, 1110, 1107, 1103, 1271, 1104, 1103, 4382, 1293, 1180, 1152, 6477, 1146, 1113, 1115, 136, 165, 183, 165, 183, 2240, 1400, 1103, 7958, 2633, 8888, 119, 18959, 2246, 2692, 1183, 1105, 1136, 11215, 1120, 1155, 1142, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [10]:
from transformers import AutoModelForSequenceClassification

#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
model = AutoModelForSequenceClassification.from_pretrained("/root/autodl-tmp/bert-base-cased-model", local_files_only=True, num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/bert-base-cased-model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [11]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [12]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [13]:
import numpy as np
import evaluate

#metric = evaluate.load("accuracy")
metric = evaluate.load("/root/autodl-tmp/evaluate/metrics/accuracy/accuracy.py")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.877700,0.903454,0.626700
2,0.794100,0.870854,0.639280
3,0.764800,0.840121,0.649160


TrainOutput(global_step=243750, training_loss=0.870986673458784, metrics={'train_runtime': 31143.8474, 'train_samples_per_second': 62.613, 'train_steps_per_second': 7.827, 'total_flos': 5.130803778048e+17, 'train_loss': 0.870986673458784, 'epoch': 3.0})

In [19]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [20]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.959382176399231,
 'eval_accuracy': 0.58,
 'eval_runtime': 0.5261,
 'eval_samples_per_second': 190.093,
 'eval_steps_per_second': 24.712,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [21]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [22]:
trainer.save_state()

## Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少